In [1]:
import tensorflow as tf
import pprint
import numpy as np

# Log in to your W&B account
!pip install wandb
import wandb
wandb.login()
# API: 18fe909a998e642a33986bd03c61ee2c6922f72d


# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train_normalized = x_train/255
x_test_normalized = x_test/255
x_train= x_train_normalized.reshape(-1, 28, 28, 1)
x_test = x_test_normalized.reshape(-1, 28, 28, 1)

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=f82851b0af1ad9d02f69889a0882e4a238a5cc7825a65027ede2e89cede369fc
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


11490434/11490434 [==============================] - 1s 0us/step
x_train: (60000, 28, 28, 1)
y_train: (60000,)
x_test: (10000, 28, 28, 1)
y_test: (10000,)


In [4]:
def Net(neurons1=26, neurons2=26, neurons3=54, acivation1='relu',activation2='relu',activation3='sigmoid',do1=0.2, do2=0.2, do3=0.2, do4=0.2, gn1=0.1):
    return tf.keras.models.Sequential(
    [
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(filters=neurons1, kernel_size=5, padding='same', activation=acivation1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(do1),
    tf.keras.layers.Conv2D(filters=neurons2, kernel_size=5, padding='same', activation=activation2, kernel_initializer=tf.keras.initializers.HeNormal()), #new
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(do2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(do3),
    tf.keras.layers.Dense(neurons3, kernel_regularizer = tf.keras.regularizers.l2(0.07), activation = activation3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(do4),
    tf.keras.layers.GaussianNoise(gn1),
    tf.keras.layers.Dense(10, activation='softmax', use_bias=False)
    ]
)

model = Net()
model.summary()

def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)

def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq='epoch'):

    # Compile model like you usually do.
    tf.keras.backend.clear_session()

    # Compile modell
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy() #from_logits=True
    sampleID = 100
    loss_fn(y_train[:1], model(x_train[sampleID-1:sampleID]).numpy()).numpy()

    model.compile(loss=loss_fn,
                  optimizer=get_optimizer(lr, optimizer),
                  metrics=['accuracy'])

    # callback setup
    wandb_callbacks = [wandb.keras.WandbCallback(log_freq=log_freq,save_model=(False),) ]

    # Train your model
    model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data = (x_test, y_test),
        callbacks = [wandb_callbacks]
        )

# Configure Sweep
sweep_config = {
    'method': 'random',
    'metric': {
        'goal': 'maximize',
        'name': 'val_accuracy'
    }
}
parameters_dict = {
    'optimizer': {
        'value': 'adam'
        },
    'epochs': {
        'value': 70
        },
    'batch_size' : {
        'values': [32,64]
        },
    'learning_rate': {
        'values': [5e-5,1e-4,2e-4]
    },
     'neurons1': {
        'value': 28
    },
    'neurons2': {
        'value': 28
    },
    'neurons3': {
        'value': 54
    },

    'activation1' : {
        'value' :  ['relu']
    },
    'activation2' : {
        'value' : ['relu']
    },
    'activation3' : {
        'value' : ['sigmoid']
    },
    'do1' : {
        'value' : 0.5
    },
    'do2' : {
        'value' : 0.4
    },
    'do3' : {
        'value' : 0.1
    },
    'do4' : {
        'value' : 0.40
    },
    'gn1' : {
        'value' : 0.7
    }
}
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "V03c"
        wandb.config.dataset_name = "MNIST"

        # initialize model
        model = Net(wandb.config.neurons1, wandb.config.neurons2, wandb.config.neurons3)

        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)

sweep_id = wandb.sweep(sweep_config, project="V03_Dropout")
wandb.agent(sweep_id, function=sweep_train, count=30)

582/938 [=================>............] - ETA: 2s - loss: 1.8624 - accuracy: 0.9345Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 26)        676       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 26)        104       
586/938 [=================>............] - ETA: 2s - loss: 1.8621 - accuracy: 0.9345 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 26)        0         
 D)                                                              
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 26)        104       
 chNormalization)                                                
                    

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


 6c2mwb55
Sweep URL: https://wandb.ai/luchalt/V03_Dropout/sweeps/6c2mwb55
938/938 [==============================] - 8s 8ms/step - loss: 1.8522 - accuracy: 0.9353 - val_loss: 1.7156 - val_accuracy: 0.9676
Epoch 53/200
846/938 [==========================>...] - ETA: 0s - loss: 1.8067 - accuracy: 0.9346

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 285, in check_stop_status
    self._loop_check_status(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 223, in _loop_check_status
    local_handle = request()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/interface/interface.py", line 727, in deliver_stop_status
    return self._deliver_stop_status(status)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/interface/interface_shared.py", line 459, in _deliver_stop_status
Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/th

850/938 [==========================>...] - ETA: 0s - loss: 1.8065 - accuracy: 0.9346

...

wandb: 	do1: 0.5
wandb: 	do2: 0.4
wandb: 	do3: 0.1
wandb: 	do4: 0.4
wandb: 	epochs: 70
wandb: 	gn1: 0.7
wandb: 	learning_rate: 0.0001
wandb: 	neurons1: 28
wandb: 	neurons2: 28
wandb: 	neurons3: 54
wandb: 	optimizer: adam


932/938 [============================>.] - ETA: 0s - loss: 1.8042 - accuracy: 0.9346

Traceback (most recent call last):
  File "<ipython-input-3-f0c1dd832531>", line 129, in sweep_train
    train(model,
  File "<ipython-input-3-f0c1dd832531>", line 50, in train
    model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/wandb/integration/keras/keras.py", line 611, in on_epoch_end
    wandb.log({"epoch": epoch}, commit=False)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 419, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 370, in wrapper_fn
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 360, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 1792, in log

Error in callback <function _WandbInit._pause_backend at 0x7b23101ed900> (for post_run_cell):


BrokenPipeError: ignored

In [ ]:
# after the training finishes, we will also save Marvin in Keras style (HDF5), so we do not have to
# train him again
# every time we start our computer. Obviously, by changing the model_name, you can also save different
# configurations of Marvin. The name has to be a string, like this: 'name.h5'
model_name = 'pepsi_p62k_a9915'
model.save(model_name, save_format='h5')

# It is best practice to indicate what configuration changes you did within the name, so you know
# which model you need to load already from its name
# Let's say instead of a learning rate of 0.001 you used 0.1, your naming could then look like:
# 'marvin_lr01.h5'

print('Success! You saved Marvin as: ', model_name)

In [ ]:
# load a saved marvin configuration you want to evaluate
model_name = 'pepsi_lr03'
pepsi_reloaded = tf.keras.models.load_model(model_name)

# Let Marvin predict on the test set, so we have some data to evaluate his performance.
predictions = pepsi_reloaded.predict([x_test])
# predictions = pepsi.predict([x_test])

# Remember that the prediction of Marvin is a probability distribution over all ten-digit classes
# We want him to assign the digit class with the highest probability to the sample.
predictions = np.argmax(predictions, axis=1)
#pd.DataFrame(predictions)

# Plot for the intuitive approach

numbers_to_display = 196
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(15, 15))

for plot_index in range(numbers_to_display):
    predicted_label = predictions[plot_index]
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    color_map = 'Greens' if predicted_label == y_test[plot_index] else 'Reds'
    plt.subplot(num_cells, num_cells, plot_index + 1)
    plt.imshow(x_test_normalized[plot_index].reshape((28, 28)), cmap=color_map)
    plt.xlabel(predicted_label)

plt.subplots_adjust(hspace=1, wspace=0.5)
plt.show()

In [ ]:
confusion_matrix = tf.math.confusion_matrix(y_test, predictions)

f, ax = plt.subplots(figsize=(9, 7))
sn.heatmap(
    confusion_matrix,
    annot=True,
    linewidths=.7,
    fmt="d",
    square=True,
    ax=ax,
    cmap="viridis",
)
plt.show()